### Oracle AI Vector Search wrapped as a llama-index custom Vector Store

* inspired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* updated after **OCI GenAI GA**
* updated for Llamaindex 0.10+

In this **first demo** we show:
* how to embed a Text using OCI GenAI Embeddings (Cohere V3)
* How to query the Oracle AI Vector Store
* How to create a simplified QA retriever using LlamaIndex
* How to answer the question using a LLM

In [1]:
import logging

from typing import List, Any, Optional, Dict, Tuple
from llama_index.core.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode, BaseNode, Document
from llama_index.core import Settings

import ads
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from prepare_chain_4_chat import create_chat_engine

from config import EMBED_MODEL
from config_private import COMPARTMENT_OCID

/Users/lsaetta/miniforge3/envs/rag_env02/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# this is the endpoint after GA, for now Chicago Region
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

In [3]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=api_keys_config,
    truncate="END",
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

# instantiate the client for the LLM
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    temperature=0.1,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for AI Vector Sesarch to find documents relevant to the query

In [4]:
v_store = OracleVectorStore(verbose=True)

In [5]:
question = (
    # "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
    "What is Oracle Data Guard? Can it be used for Disaster Recovery"
)

In [6]:
# embed the query using OCI GenAI

query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=6)

In [7]:
%%time

q_result = v_store.query(query_obj)

CPU times: user 24.4 ms, sys: 7.28 ms, total: 31.7 ms
Wall time: 606 ms


#### Displays results

In [8]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):

    print(n.text)
    print("")
    print(n.metadata)
    print(f"Similarity: {round(sim, 4)}")
    print("")

Part IV Oracle Data Guard Best Practices •Overview of MAA Best Practices for Oracle Data Guard •Plan an Oracle Data Guard Deployment •Configure and Deploy Oracle Data Guard •Tune and Troubleshoot Oracle Data Guard •Monitor an Oracle Data Guard Configuration 

{'file_name': 'high-availability-23c.pdf', 'page_label': '1'}
Similarity: 0.3112

Part II Oracle Database High Availability Best Practices •Overview of Oracle Database High Availability Best Practices •Oracle Database Configuration Best Practices •Oracle Flashback Best Practices 

{'file_name': 'high-availability-23c.pdf', 'page_label': '1'}
Similarity: 0.3305

7 High Availability Data Guard Oracle Data Guard Redo Decryption for Hybrid Disaster Recovery Configurations Oracle Data Guard now provides the capability to decrypt redo operations in hybrid cloud disaster recovery configurations where the cloud database is encrypted with Transparent Data Encryption (TDE) and the on-premises database is not. Hybrid disaster recovery (DR) w

#### Integrate in the RAG picture (the entire chain: Embeddings -> Vector Store -> LLM)

In [9]:
# replaced ServiceContext with Settings (llamaindex > 0.10)
Settings.embed_model = embed_model
Settings.llm = llm_oci

index = VectorStoreIndex.from_vector_store(vector_store=v_store)

query_engine = index.as_query_engine(similarity_top_k=5)

In [10]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print("")
print(response.response)
print("")

Question: What is Oracle Data Guard? Can it be used for Disaster Recovery

Oracle Data Guard is a tool that helps you to configure and maintain high availability and disaster recovery for your Oracle databases. It ensures that there is always a backup database available to take over in the event of a failure or outage. You can also use Oracle Data Guard to manage and monitor your databases to ensure that they are always available and that data is protected. 

Data Guard works by replicating and storing the changes made to a database in another database, known as the standby database. This allows for continuous access to data even if the primary database is unavailable. 

Disaster recovery with Data Guard in a hybrid environment involves using both on-premises databases and cloud databases in Oracle Cloud Infrastructure (OCI). It enables you to extend outage and data protection by leveraging the automation and resources of the cloud. 

Some best practices for using Oracle Data Guard for

#### Using the chat engine: a single line of code

In [17]:
chat_engine, token_counter = create_chat_engine(token_counter=None, verbose=False)

In [18]:
response = chat_engine.chat(question)

In [19]:
print(response)

Oracle Data Guard is a high availability feature used in Maximum Availability Architecture (MAA) solutions. It ensures availability and protection for enterprise data in the event of any outages, including natural disasters and data corruption, thus enabling disaster recovery.

Data Guard creates exact replicas, called standby databases, of primary databases, which can be used for various purposes, including disaster recovery. These standby databases receive redo logs from the primary database, enabling them to stay up-to-date and synchronize with it. In case of a disaster or outage affecting the primary database, a standby database can quickly take over, minimizing downtime and ensuring continuous operations. 

Data Guard also provides features like automatic corruption repair, reducing downtime for maintenance and upgrades, and the ability to offload certain activities to the standby database. It integrates seamlessly with other Oracle technologies, such as Oracle RAC, RMAN, and Ente

In [22]:
# if yiu want to count the token
token_counter.completion_llm_token_count

380